In [1]:
from summary_reader import read_tensors

filename = "../results/dnn-full/error_analysis/events.out.tfevents.1504982945.jarvis"
iterator = read_tensors(filename)

x = next(iterator)

In [23]:
import numpy as np

# Find errors in the batch.
idx = np.where(x['is_error'].ravel())[0]

print idx
np.sum(x['is_error'])

def examples_by_index(features, indices):
    return [dict((k, features[k][idx]) for k in features.keys()) for idx in indices]

def html_table(errors, keys=None):
    """ Returns HTML string for a table of the errors.
    
    Args:
        errors: List of dictionaries. All elements of dictionaries will be represented
                as strings in the table.
        keys: Keys from the dictionaries that should be shown. If None, will use keys
              from first element of the |errors|.

    Returns:
        Valid HTML string.
    """
    if not errors:
        return ""

    keys = keys or errors[0].keys()
    s = "<table>"
    s += "<tr><th>" + "</th><th>".join(keys) + "</th></tr>"
    row_template = "<tr><td>"+"</td><td>".join("{"+key+"}" for key in keys) + "</td></tr>"
    s += "".join(row_template.format(**error) for error in errors)
    s += "</table>"
    return s

from IPython.display import HTML, display
def show_table(errors):
    display(HTML(html_table(errors)))

show_table(examples_by_index(x, idx))

[ 0  1  3  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 22 23 24 26 27 28 30
 31 32 35 36 37 38 39 40 41 42 43 44 45 47 48 49 51 52 53 56 57 58 59 60 61
 62 63 64 65 67 68 70 73 74 75 78 79 80 82 83 84 85 86 87 88 89 90 91 93 94
 95 97]


BillTitle,is_error,VoterState,SponsorParty,prediction,VoterParty,VoterAge
[ 4 865 1 1330 34 3 452 5 1333 1354 1365 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['WA'],['UNK'],[ True],['republican'],[66]
[ 11 10 2 90 72 3 197 41 2 1 8 9 14 15 16 126 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['NJ'],['democrat'],[ True],['democrat'],[66]
[ 4 13 36 737 5 1 143 48 12 4 68 3 44 1 464 30 79 52 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['SD'],['democrat'],[ True],['democrat'],[46]
[ 28 12 4 13 1 979 288 12 5 710 4 280 1 21 20 979 34 46 94 568 17 3 3 731 5 296 704 3 70 33 17 1 21 20 3 4 155 335 3 35 186 1033 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['NC'],['republican'],[ True],['republican'],[86]
[11 74 54 10 2 1 8 9 75 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['UT'],['republican'],[ True],['republican'],[84]
[ 4 19 89 145 2 106 524 4 114 460 4 416 159 4 190 1 1795 1294 814 5 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['CA'],['republican'],[ True],['republican'],[56]
[ 11 10 2 1 151 144 2 1 8 9 14 15 16 154 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['LA'],['republican'],[ True],['republican'],[70]
[ 28 12 11 10 2 38 60 3 96 98 3 25 24 2 1 8 9 14 15 16 154 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['CA'],['republican'],[ True],['republican'],[69]
[11 10 2 1 22 5 37 2 1 8 9 14 15 16 87 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['AR'],['democrat'],[ True],['democrat'],[89]
[ 4 13 36 737 5 1 143 48 12 4 68 3 44 1 464 30 79 52 3 2 6 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],[ True],['CT'],['democrat'],[ True],['democrat'],[74]
